In [1]:
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
#VARIABLE ADJUSTMENT:

#Mass of Mimas (real world = 3.75e+19 kg):
mMimas=3.75e+23 
#exaggerated default mass for effect (3.75e+23 kg)

#Number of Particles (default 800 particles):
nParticles = 800

#Orbital distance of Mimas (real world = 1.85e+8 meters): 
dSaturn_Mimas = 1.85e+8

#Inner Ring Radius (real world B ring = 9.2e+7 meters):
B_R = 9.2e+7

#Outer Ring Radius (real world A ring = 1.37e+8 meters):
A_R = 1.37e+8

#Runtime (default = 1000, runs ~1 Mimas-orbit with default positions):
n = 1000

#Timestep (default = 25):
tstep = 25 

#Do not recommend changing:

#Mass of Saturn (real world = 5.69e+26 kg):
mSat=5.69e+26

#Mass per particle (default = 1, particles only react to Mimas and Saturn and do not exert any gravitational pull)
mParticles = 1

#Gravitational Constant:
G=6.67e-11

#ODE Timestep as a fraction of orbital period (default = 1000, as in 1/1000 of Mimas' orbital period)
step_fraction = 1000

In [3]:
rSat_rCM=dSaturn_Mimas*mMimas/(mMimas+mSat)

rSat=np.array([-rSat_rCM,0])
rMimas=np.array([dSaturn_Mimas-rSat_rCM,0])

mSat=5.69*10**26
angular_v=np.sqrt(G*(mSat+mMimas)/np.linalg.norm(rMimas-rSat)**3)
mag_vMimas = angular_v*np.linalg.norm(rMimas)
mag_vSat= angular_v*np.linalg.norm(rSat)

vMimasx=0
vMimasy=mag_vMimas

vMimas=np.array([vMimasx,vMimasy])

vSatx=0
vSaty=-mag_vSat

vSat=np.array([vSatx,vSaty])

y=np.concatenate([rSat,vSat,rMimas,vMimas])
print(y)

print(angular_v)
print(vSat,vMimas)

[-1.21844127e+05  0.00000000e+00  0.00000000e+00 -9.43646043e+00
  1.84878156e+08  0.00000000e+00  0.00000000e+00  1.43182560e+04]
7.744698608954474e-05
[ 0.         -9.43646043] [    0.         14318.25596613]


In [4]:
#PARTICLE INITIAL POSITIONS AND VELOCITIES
count=0 
rParticles = []
xdata = []
ydata = []
while count < nParticles:
    r_radius = np.random.random()
    r_theta  = np.random.random()
    radius = B_R + (A_R - B_R) * r_radius
    theta = r_theta * 2 * np.pi
    x = np.cos(theta)*radius
    y = np.sin(theta)*radius
    rParticles.append([x,y])
    xdata.append(x)
    ydata.append(y)
    count+=1

vParticles = []

for i in rParticles:
    angular_v=np.sqrt(G*(mSat+mMimas)/np.linalg.norm(i-rSat)**3)
    mag_vParticle = angular_v*np.linalg.norm(i)
    xVelocity = -angular_v * i[1]
    yVelocity = angular_v * i[0]
        
    vParticles.append([xVelocity,yVelocity])

z=np.concatenate([rParticles,vParticles])
print(rParticles[105])
print(vParticles[105])
rParticles = np.array(rParticles)
vParticles = np.array(vParticles)

[30693458.46926409, -125094777.7734023]
[16670.70074349976, 4090.3502930474356]


In [5]:
#DEFINING DIFFERENTIAL EQUATIONS 
Psquare=(4*np.pi**2)*((dSaturn_Mimas-rSat_rCM)**3)/(G*(mSat))
P=np.sqrt(Psquare)
print('T=',P)
dt=P/step_fraction

t=0

def MimasODE(t,y):
    global G,mMimas,mSat
    rSat =y[0:2]
    vSat =y[2:4]
    rMimas =y[4:6] 
    vMimas =y[6:8] 

    drsdt=vSat
    drmdt=vMimas
    
    Fsat_mimas=-(rSat-rMimas)*G*mSat*mMimas/np.linalg.norm(rSat-rMimas)**3
   

    Fmimas_sat=-(rMimas-rSat)*G*mMimas*mSat/np.linalg.norm(rMimas-rSat)**3
  
    a_sat = (Fsat_mimas)/mSat
    
    a_mimas = (Fmimas_sat)/mMimas

    #dvdt = a 
    
    dvsdt = a_sat
    dvmdt = a_mimas

    return np.concatenate((drsdt,dvsdt,drmdt,dvmdt))

def ParticlesODE(t,z):
    global G,mMimas,mSat,mParticles
    
    rParticle = z[0:2]
    vParticle = z[2:4]
    
    drdt = vParticle
    
    Fparticle_saturn=-(rParticle-rSat)*G*mParticles*mSat/np.linalg.norm(rParticle-rSat)**3
    Fparticle_mimas=-(rParticle-rMimas)*G*mParticles*mMimas/np.linalg.norm(rParticle-rMimas)**3
    aParticle = (Fparticle_saturn + Fparticle_mimas)/mParticles
    
    dvdt = aParticle
    
    return np.concatenate((drdt,dvdt))

y=np.concatenate([rSat,vSat,rMimas,vMimas])
y0 = np.copy(y)
print(MimasODE(0,y))
print(y0)

T= 81075.42219735048
[ 0.00000000e+00 -9.43646043e+00  7.30825420e-04 -0.00000000e+00
  0.00000000e+00  1.43182560e+04 -1.10890577e+00 -0.00000000e+00]
[-1.21844127e+05  0.00000000e+00  0.00000000e+00 -9.43646043e+00
  1.84878156e+08  0.00000000e+00  0.00000000e+00  1.43182560e+04]


In [6]:
rParticles0 = np.copy(rParticles)
vParticles0 = np.copy(vParticles)

In [7]:
#ANIMATION
from matplotlib.animation import FFMpegWriter
%matplotlib qt
plt.show()
metadata = dict(title='Saturn, Mimas, and the A and B rings', artist='Brian Harris',comment='Single Orbit')
writer = FFMpegWriter(fps=15, metadata=metadata)
fig = plt.figure()
y = y0
rParticles = np.copy(rParticles0)
vParticles = np.copy(vParticles0)
print(vParticles)
for step in range(1,int(n)):
    
    rSat=y[0:2]
    rMimas=y[4:6]
    xSat = rSat[0]
    ySat = rSat[1]
    xMimas = rMimas[0]
    yMimas = rMimas[1]

    f1 = MimasODE(t       ,y          )
    f2 = MimasODE(t+dt/2.0,y+f1*dt/2.0)
    f3 = MimasODE(t+dt/2.0,y+f2*dt/2.0)
    f4 = MimasODE(t+dt    ,y+f3*dt    )

    y = y + (f1 + 2.0*f2 + 2.0*f3 + f4) / 6.0 * dt
    
    for r in range(len(rParticles)):
        rParticle = rParticles[r]
        vParticle = vParticles[r] 
        z=np.concatenate((rParticle,vParticle))

        g1 = ParticlesODE(t       ,z          )
        g2 = ParticlesODE(t+dt/2.0,z+g1*dt/2.0)
        g3 = ParticlesODE(t+dt/2.0,z+g2*dt/2.0)
        g4 = ParticlesODE(t+dt    ,z+g3*dt    )
        
        z += (g1 + 2.0*g2 + 2.0 * g3 + g4) / 6.0 * dt
        
        rParticles[r] = z[0:2]
        vParticles[r] = z[2:4]
        
    t += dt
    if step%tstep==0:
        with writer.saving(fig, "animation1.mp4", dpi=200):
            plt.clf()
            plt.scatter(rParticles[:, 0],rParticles[:, 1], s=3,color='gold')
            plt.plot(xSat,ySat, 'or',ms=50)
            plt.plot(xMimas,yMimas, 'ob',ms=10)
            plt.xlim(-1.4 * dSaturn_Mimas, 1.4 * dSaturn_Mimas) 
            plt.ylim(-1.4 * dSaturn_Mimas, 1.4 * dSaturn_Mimas)
            plt.xlabel('m')
            plt.ylabel('m')
            plt.draw()
            plt.pause(0.005)
            writer.grab_frame()

[[ 17703.4175854    3627.86945065]
 [-11813.08009088  12974.93549786]
 [-16735.54461951   9560.21145016]
 ...
 [-12252.20507391 -14554.09242158]
 [-15844.30841261  -8578.86839792]
 [-19464.41530761  -4583.48885125]]


KeyboardInterrupt: 